### Please use this code as a notebook at Kaggle https://www.kaggle.com/bigquery/ethereum-blockchain

In [1]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """
SELECT 
   contracts.address, COUNT(1) AS tx_count
FROM
  `bigquery-public-data.crypto_ethereum.contracts` AS contracts
JOIN
  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
ON (transactions.to_address = contracts.address)
GROUP BY contracts.address
ORDER BY tx_count DESC 
"""

Using Kaggle's public dataset BigQuery integration.


In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "crypto_ethereum" dataset 
dataset_ref = client.dataset("crypto_ethereum", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "crypto_ethereum" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

Using Kaggle's public dataset BigQuery integration.
balances
blocks
contracts
logs
token_transfers
tokens
traces
transactions


In [3]:
query_job = client.query(query)

iterator = query_job.result(timeout=300)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10
df.head(10)

,address,tx_count
0,0xdac17f958d2ee523a2206206994597c13d831ec7,20296313
1,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,11418019
2,0x174bfa6600bf90c885c7c01c7031389ed1461ab9,9087835
3,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,8575129
4,0x06012c8cf97bead5deae237070f9587f8e7a266d,4539895
5,0xd1ceeeeee83f8bcf3bedad437202b6154e9f5405,3097506
6,0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0,2964819
7,0xf230b790e05390fc8295f4d3f60332c93bed42e2,2069815
8,0x8fdcc30eda7e94f1c12ce0280df6cd531e8365c5,1926315
9,0x209c4784ab1e8183cf58ca33cb740efbf3fc18ef,1814557


In [4]:
df.to_csv("ethereum_contracts_with_transaction_amount.csv", index=False)